In [ ]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, SimpleRNN, LSTM
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt



In [ ]:
!pip install firebase-admin

In [ ]:
import firebase_admin
from firebase_admin import credentials, storage
import os
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive') # Used to store JSON key

Mounted at /content/drive


In [ ]:
# Initialize Firebase

json_path = '### Update with the path to your JSON key ###'

cred = credentials.Certificate(json_path)  # Update with the path to your JSON key
firebase_admin.initialize_app(cred, {'storageBucket': '### Project ID ###'})  # Update with your project ID
bucket = storage.bucket()

# Define Firebase folder paths
folders = ["pneumonia", "normal"]  # Update with your folder names

In [ ]:
# List all files in the specified folder
folder_path = 'normal/'
local_image_path = '/content/normal/'

os.makedirs(local_image_path, exist_ok=True)

blobs = bucket.list_blobs(prefix=folder_path)
normal_count = 0
print(f"Files in folder '{folder_path}':")
for blob in blobs:
    print(blob.name)
    # Extract filename from blob name and join with local directory path
    filename = os.path.basename(blob.name)  # Get filename from blob.name
    full_local_path = os.path.join(local_image_path, filename) # Concatenate local directory path and filename
    blob.download_to_filename(full_local_path) # Download to the specific file path
    normal_count += 1

Files in folder 'normal/':
normal/IM-0001-0001.jpeg
normal/IM-0003-0001.jpeg
normal/IM-0005-0001.jpeg
normal/IM-0006-0001.jpeg
normal/IM-0007-0001.jpeg
normal/IM-0009-0001.jpeg
normal/IM-0010-0001.jpeg
normal/IM-0011-0001-0001.jpeg
normal/IM-0011-0001-0002.jpeg
normal/IM-0011-0001.jpeg
normal/IM-0013-0001.jpeg
normal/IM-0015-0001.jpeg
normal/IM-0016-0001.jpeg
normal/IM-0017-0001.jpeg
normal/IM-0019-0001.jpeg
normal/IM-0021-0001.jpeg
normal/IM-0022-0001.jpeg
normal/IM-0023-0001.jpeg
normal/IM-0025-0001.jpeg
normal/IM-0027-0001.jpeg
normal/IM-0028-0001.jpeg
normal/IM-0029-0001.jpeg
normal/IM-0030-0001.jpeg
normal/IM-0031-0001.jpeg
normal/IM-0033-0001-0001.jpeg
normal/IM-0033-0001-0002.jpeg
normal/IM-0033-0001.jpeg
normal/IM-0035-0001.jpeg
normal/IM-0036-0001.jpeg
normal/IM-0037-0001.jpeg
normal/IM-0039-0001.jpeg
normal/IM-0041-0001.jpeg
normal/IM-0043-0001.jpeg
normal/IM-0045-0001.jpeg
normal/IM-0046-0001.jpeg
normal/IM-0049-0001.jpeg
normal/IM-0050-0001.jpeg
normal/IM-0059-0001.jpeg
nor

In [ ]:
# List all files in the specified folder
folder_path = 'pneumonia/'
local_image_path = '/content/pneumonia/'

os.makedirs(local_image_path, exist_ok=True)

blobs = bucket.list_blobs(prefix=folder_path)
print(f"Files in folder '{folder_path}':")
pneumonia_count = 0
for blob in blobs:
    print(blob.name)
    # Extract filename from blob name and join with local directory path
    filename = os.path.basename(blob.name)  # Get filename from blob.name
    full_local_path = os.path.join(local_image_path, filename) # Concatenate local directory path and filename
    blob.download_to_filename(full_local_path) # Download to the specific file path
    pneumonia_count += 1

Files in folder 'pneumonia/':
pneumonia/.DS_Store
pneumonia/person1000_bacteria_2931.jpeg
pneumonia/person1000_virus_1681.jpeg
pneumonia/person1001_bacteria_2932.jpeg
pneumonia/person1002_bacteria_2933.jpeg
pneumonia/person1003_bacteria_2934.jpeg
pneumonia/person1003_virus_1685.jpeg
pneumonia/person1004_bacteria_2935.jpeg
pneumonia/person1004_virus_1686.jpeg
pneumonia/person1005_bacteria_2936.jpeg
pneumonia/person1005_virus_1688.jpeg
pneumonia/person1006_bacteria_2937.jpeg
pneumonia/person1007_bacteria_2938.jpeg
pneumonia/person1007_virus_1690.jpeg
pneumonia/person1008_bacteria_2939.jpeg
pneumonia/person1008_virus_1691.jpeg
pneumonia/person1009_virus_1694.jpeg
pneumonia/person100_bacteria_475.jpeg
pneumonia/person100_bacteria_477.jpeg
pneumonia/person100_bacteria_478.jpeg
pneumonia/person100_bacteria_479.jpeg
pneumonia/person100_bacteria_480.jpeg
pneumonia/person100_bacteria_481.jpeg
pneumonia/person100_bacteria_482.jpeg
pneumonia/person100_virus_184.jpeg
pneumonia/person1010_bacteria_

In [ ]:
# Access images to find their size
import cv2  # OpenCV for image processing
import numpy as np
local_paths = {'pneumonia': '/content/pneumonia/', 'normal': '/content/normal/'}

# Function to check image blurriness
def is_blurry(image, threshold=100):
    laplacian_var = cv2.Laplacian(np.array(image), cv2.CV_64F).var()  # Calculate the variance of Laplacian
    return laplacian_var < threshold

# Access images to find their size and blurriness
for folder, local_image_path in local_paths.items():
    print(f"\nImage details in '{folder}' folder:")
    for filename in os.listdir(local_image_path):
        # Skip hidden files
        if filename.startswith('.'):
            continue
        image_path = os.path.join(local_image_path, filename)
        with Image.open(image_path) as img:
          # Get image width and height
          width, height = img.size
          # Check for image blurriness
          blurry = is_blurry(img)
          if(blurry):
            blurry_or_not = "Blurry"
          else:
            blurry_or_not = "Not Blurry"
        print(f"{filename}: {width}x{height} pixels, {blurry_or_not}")

Streaming output truncated to the last 5000 lines.
person588_bacteria_2422.jpeg: 1120x656 pixels, Not Blurry
person303_bacteria_1431.jpeg: 1562x1106 pixels, Blurry
person1157_virus_1937.jpeg: 1152x720 pixels, Blurry
person1594_virus_2768.jpeg: 1272x776 pixels, Not Blurry
person427_bacteria_1868.jpeg: 968x568 pixels, Not Blurry
person924_bacteria_2849.jpeg: 920x752 pixels, Not Blurry
person258_bacteria_1214.jpeg: 1088x840 pixels, Not Blurry
person124_virus_245.jpeg: 1184x1048 pixels, Not Blurry
person1616_virus_2802.jpeg: 1288x920 pixels, Not Blurry
person255_bacteria_1182.jpeg: 1056x632 pixels, Not Blurry
person331_virus_684.jpeg: 1184x752 pixels, Not Blurry
person1589_bacteria_4171.jpeg: 1136x712 pixels, Not Blurry
person338_bacteria_1568.jpeg: 1240x752 pixels, Not Blurry
person1645_bacteria_4363.jpeg: 1152x664 pixels, Not Blurry
person1349_bacteria_3439.jpeg: 1504x992 pixels, Blurry
person407_bacteria_1822.jpeg: 1106x650 pixels, Not Blurry
person255_bacteria_1162.jpeg: 920x544 pixels

In [ ]:
#Class Imbalances

# Get the number of images in each class
normal_count = len(os.listdir('/content/normal/'))
pneumonia_count = len(os.listdir('/content/pneumonia/'))

print("\nClass Distribution Summary:")
print(f"Class Normal: {normal_count} images")
print(f"Class Pneumonia: {pneumonia_count} images")

# Check for class imbalance
total_images = pneumonia_count + normal_count
normal_percentage = (normal_count / total_images) * 100
pneumonia_percentage = (pneumonia_count / total_images) * 100
print(f"Class Normal accounts for {normal_percentage:.2f}% of the dataset")
print(f"Class Pneumonia accounts for {pneumonia_percentage:.2f}% of the dataset")


Class Distribution Summary:
Class Normal: 1583 images
Class Pneumonia: 4274 images
Class Normal accounts for 27.03% of the dataset
Class Pneumonia accounts for 72.97% of the dataset


In [ ]:
# Handle class imbalances by undersampling larger class
import random
import shutil
# Create folder for undersampled pneumonia images
os.makedirs('/content/undersampled_pneumonia/', exist_ok=True)

# Randomly select a subset of pneumonia images
undersampled_pneumonia_images = random.sample(os.listdir('/content/pneumonia/'), normal_count)

# Copy the undersampled pneumonia images to the new directory
for img in undersampled_pneumonia_images:
  shutil.copy(os.path.join('/content/pneumonia/', img), '/content/undersampled_pneumonia/')

print(f"Undersampled pneumonia class to {normal_count} images.")

Undersampled pneumonia class to 1583 images.


In [ ]:
# Import necessary libraries for Vision Transformer (ViT)
import os
import random
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import torch
from transformers import ViTForImageClassification

# Define custom dataset for the images
class ChestXRayDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = 0 if "normal" in img_path else 1  # Assuming folder name indicates class
        if self.transform:
            image = self.transform(image)
        return image, label

# Image paths for dataset creation
normal_images = [os.path.join('/content/normal/', img) for img in os.listdir('/content/normal/')]
pneumonia_images = [os.path.join('/content/undersampled_pneumonia/', img) for img in os.listdir('/content/undersampled_pneumonia/')]
all_images = normal_images + pneumonia_images

# Splitting the data into train (60%), validation (20%), and test (20%) sets
train_images, temp_images = train_test_split(all_images, test_size=0.4, random_state=42, stratify=[0 if "normal" in img else 1 for img in all_images])
val_images, test_images = train_test_split(temp_images, test_size=0.5, random_state=42, stratify=[0 if "normal" in img else 1 for img in temp_images])

# Data transforms for ViT
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Creating datasets and dataloaders
train_dataset = ChestXRayDataset(train_images, transform=transform)
val_dataset = ChestXRayDataset(val_images, transform=transform)
test_dataset = ChestXRayDataset(test_images, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2)


In [ ]:
# Loading Vision Transformer (ViT) model
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224', num_labels=2, ignore_mismatched_sizes=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Updated training loop
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(pixel_values=images).logits
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Training Loss: {total_loss/len(train_loader)}")

    # Validation step
    model.eval()
    correct = 0
    total = 0
    val_loss = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(pixel_values=images).logits
            val_loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    val_accuracy = 100 * correct / total
    print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

Epoch 1/5, Training Loss: 0.27156242114460455
Validation Loss: 0.2104, Validation Accuracy: 95.26%
Epoch 2/5, Training Loss: 0.16949087169369637
Validation Loss: 0.1316, Validation Accuracy: 96.05%
Epoch 3/5, Training Loss: 0.07806379711886421
Validation Loss: 0.1595, Validation Accuracy: 96.21%
Epoch 4/5, Training Loss: 0.049516557615070345
Validation Loss: 0.2836, Validation Accuracy: 94.63%
Epoch 5/5, Training Loss: 0.02981705255275729
Validation Loss: 0.2679, Validation Accuracy: 94.63%


In [ ]:
# Save the model state dict
torch.save(model.state_dict(), 'vit_model.pth')

In [ ]:
print(model)

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fe

Using ViTForImageClassification model, which is pretrained on ImageNet dataset. Adapted this model for binary classification task (num_labels=2).

Uses GPU(CUDA) if possible, otherwise runs on CPU.

AdamW Optimizer:
- AdamW is generally well-suited for transformer-based models like ViT due to its ability to decouple weight decay and learning rate. This helps improve generalization.
In combination with a moderate learning rate like 1e-4, it tends to perform well for fine-tuning tasks.

CrossEntropyLoss used for classification, very effective for binary classification.

Model Training:

- forward pass = input images going thru model to get outputs, which have the predictions before applying softmax

- then, difference between predicted outputs and actual labels is calculated with CrossEntropyLoss

- backward pass:
    
    - loss.backward calculates gradients for each parameter
    - optimizer.zero_grad() resets gradients from previous iteration
    - torch.nn.utils.clip_grad_norm_() caps gradients to a max value of 1.0
    - optimizer.step() updates model parameters based on computed gradients.

Validation:

- same forward pass as model training
- evaluates model on the validation set without updating weights
- logs training and validation losses, model accuracy

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score

# Set the model to evaluation mode
model.eval()

# Initialize variables to track performance metrics
correct = 0
total = 0
test_loss = 0

all_labels = []
all_predictions = []

# Disable gradient calculation for testing
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(pixel_values=images).logits
        test_loss += criterion(outputs, labels).item()  # Accumulate loss

        # Get predicted class
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        #store predictions and labels for metrics analysis
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate and print test accuracy and loss
test_loss /= len(test_loader)  # Average test loss
test_accuracy = 100 * correct / total  # Test accuracy percentage
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')


Test Loss: 0.2439, Test Accuracy: 95.27%


In [ ]:
# Calculate and display additional metrics
print("Classification Report:")
print(classification_report(all_labels, all_predictions, target_names=["Normal", "Pneumonia"]))

Classification Report:
              precision    recall  f1-score   support

      Normal       0.94      0.97      0.95       317
   Pneumonia       0.96      0.94      0.95       317

    accuracy                           0.95       634
   macro avg       0.95      0.95      0.95       634
weighted avg       0.95      0.95      0.95       634



Gradients are disabled with torch.no_grad() since gradients are not needed during inference


In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
# Calculate and display ROC-AUC score
roc_auc = roc_auc_score(all_labels, all_predictions)
print(f'ROC-AUC Score: {roc_auc:.4f}')

# Loading the Model

In [ ]:
import torch
from transformers import ViTForImageClassification

model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224', num_labels=2, ignore_mismatched_sizes=True)
# Load the model state dict
model.load_state_dict(torch.load('vit_model.pth', map_location=torch.device('cpu')))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.eval()  # Set the model to evaluation mode if you are using it for inference